Pontifícia Universidade Católica de São Paulo 

`Ciência de Dados e Inteligência Artificial`

#🎓 Aula 5b - Support Vector Machines
09 de setembro de 2021


---
> 👨‍🏫*Professor Rooney Coelho (rracoelho@pucsp.br)*
---




Neste aula você usará SVM (Support Vector Machines) para construir e treinar um modelo usando registros de células humanas e classificar as células se as amostras são benignas ou malignas.

O SVM funciona mapeando dados para um espaço de recursos de alta dimensão para que os pontos de dados possam ser categorizados, mesmo quando os dados não são linearmente separáveis. Um separador entre as categorias é encontrado, então os dados são transformados de tal forma que o separador pode ser desenhado como um hiperplano. Em seguida, as características dos novos dados podem ser usadas para prever o grupo ao qual um novo registro deve pertencer.

In [1]:
# Dependências
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split

### Importação e limpeza dos dados

1) Importe os dados

In [5]:
df = pd.read_csv("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/cell_samples.csv")
df

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


O campo ID contém os identificadores do paciente. As características das amostras de células de cada paciente estão contidas nos campos Clump à Mit. Os valores são classificados de 1 a 10, sendo 1 o mais próximo de benigno.

O campo Class contém o diagnóstico, conforme confirmado por procedimentos médicos separados, se as amostras são benignas (valor = 2) ou malignas (valor = 4).

Vejamos a distribuição das classes com base Clump thickness e Uniformity of cell size:

2) Use o método `info()` para chechar quais colunas possuem valores não numéricos.

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           699 non-null    int64 
 1   Clump        699 non-null    int64 
 2   UnifSize     699 non-null    int64 
 3   UnifShape    699 non-null    int64 
 4   MargAdh      699 non-null    int64 
 5   SingEpiSize  699 non-null    int64 
 6   BareNuc      699 non-null    object
 7   BlandChrom   699 non-null    int64 
 8   NormNucl     699 non-null    int64 
 9   Mit          699 non-null    int64 
 10  Class        699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


Note que a coluna `BareNuc`possui valores não numéricos. Vamos usar agora o método `unique()`nesta coluna para ver quais são os valores únicos dessa coluna.

In [13]:
df['BareNuc'].unique()

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

3) Veja que os valores faltantes na coluna `BareNuc`são iguais a `'?'`. Use o método `replace()` em `df` para substituir `'?'` por `np.NaN`. Sobreescreva `df`para não perder a alteração realizada.

In [19]:
df['BareNuc'] = df['BareNuc'].replace(['?'], np.NaN)

4) Use o método `dropna`do Pandas para remover todas as linhas que contenham `np.NaN`. Sobreescreva o dataframe para efetivar as alterações.

In [23]:
df = df.dropna()

### Identificação das features e target

4) Crie um DataFrame `X` com todas as features. Lembrando que o target é o atributo Class e que ID não é uma feature.

In [25]:
X = df.drop('Class',axis=1)
X.head()

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit
0,1000025,5,1,1,1,2,1,3,1,1
1,1002945,5,4,4,5,7,10,3,2,1
2,1015425,3,1,1,1,2,2,3,1,1
3,1016277,6,8,8,1,3,4,3,7,1
4,1017023,4,1,1,3,2,1,3,1,1


5) Segmente o Dataframe para a representação do Target (Class)

In [27]:
y = df[['Class']]
y

,Class
0,2
1,2
2,2
3,2
4,2
...,...
694,2
695,2
696,4
697,4


### Separação dos dados para teste e validação.

6) Divida os dados em treino e teste com a função `train_test_split()`. Separe 20% dos dados para validação.

In [30]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X,y, test_size=0.2)

Verifique se as dimensões dos dados de teste e validação estão coerentes.

In [31]:
len(X_treino+X_teste) == len(df)

True

In [59]:
len(X_treino) == len(y_treino)

True

### Treinamento e avaliação da SVM criada

O algoritmo SVM oferece uma escolha de funções de kernel para realizar seu processamento. Basicamente, o mapeamento de dados em um espaço dimensional superior é chamado de kernelling. A função matemática usada para a transformação é conhecida como função kernel e pode ser de diferentes tipos, como:

     1. Linear
     2. Polinômio
     3. Função de base radial (RBF)
     4.Sigmóide
Cada uma dessas funções tem suas características, seus prós e contras e sua equação, mas como não há uma maneira fácil de saber qual função tem o melhor desempenho com qualquer dataset, geralmente escolhemos funções diferentes e comparamos os resultados. 

7) Utilize a Função de base radial (RBF) como kernel. O classificador SVM é representado pelo método `SVC`

In [62]:
modelo = svm.SVC(kernel = 'rbf')
modelo.fit(X_treino,y_treino)

c:\users\adriana\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC()

8) Use o método `score()` para avaliar a performance de seu classificador.

In [63]:
modelo.score(X_teste,y_teste)

0.656934306569343